In [1]:
using QuantumOptics

In [2]:
"""
Compute the single-particle operator from single-particle matrix.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `sp_matrix::Matrix`: Hopping phases matrix from any model.
"""
function get_sp_op(sp_basis, sp_matrix)
    
    H = SparseOperator(sp_basis)

    N, = size(sp_matrix)
    
    for m in 1:N
        for n in 1:N
            H += sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op

In [3]:
"""
Compute the first eigen-states in a given number.

#### Arguments
- `sp_op::get_sp_op`: Single-particle operator.
- `cut_off::Int`: Limit value of eigen-state index.
"""
function get_subset_states(sp_op, sub_range)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[sub_range]
    
    return states
end

get_subset_states

In [14]:
"""
Compute sub-space basis, projection and comlex conjugate of projection operator.

#### Arguments
- `states::get_sub_states`: Eigen-states of the sub-space.
- `basis::NLevelBasis`: Single-particle basis.
"""
function get_projector_op(basis, proj_basis)
    
    subspace_proj = SubspaceBasis(basis, proj_basis)
    P = projector(subspace_proj, basis)
    Pt = dagger(P)
    
    return subspace_proj, P, Pt
end

get_projector_op

In [5]:
"""
Compute the corresponding operator in the sub-space.

#### Arguments
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
- `P::get_projector_op[2]`: Projection operator.
- `Pt::get_projector_op[3]`: Complex conjugate of projection operator.
"""
function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end

get_subspace_op

In [11]:
"""
Compute the many-body operator for boson particles from single-particle operator.

#### Arguments
- `mb_basis`: Many-body basis.
- `sp_op::Operator`: Single-particle operator.
"""
function get_mb_op(mb_basis, sp_op)
    
    mb_op = SparseOperator(mb_basis)

    N = size(sp_op)[1]
    
    for i in 1:N
        for j in 1:N
            mb_op += sp_op.data[i,j] * transition(mb_basis, i, j)
        end
    end
    
    return mb_op
end

get_mb_op

In [12]:
function get_H_mb(param_dict)
    H1_m = param_dict["model_H1"](param_dict)    
    H1 = get_sp_op(param_dict["basis"], H1_m)
    if "sub_range" in keys(param_dict)
        #sub_states = get_subset_states(H1, param_dict["sub_range"]);
        #subspace_proj, P, Pt = get_projector_op(param_dict["basis"], sub_states)
        #param_dict["subspace_proj"] = subspace_proj
        subspace_proj, P, Pt = param_dict["projection"]
        H1_proj = get_subspace_op(H1, P, Pt)
        #basis_mb = get_basis_mb(param_dict)
        H_mb = get_mb_op(param_dict["basis_mb"], H1_proj)
        H_mb = (H_mb'+H_mb)/2
    else
        basis_mb = get_basis_mb(param_dict)
        H_mb = get_mb_op(param_dict["basis_mb"], H1)
    end
    return H_mb
end

get_H_mb (generic function with 1 method)

In [13]:
function get_basis_mb(param_dict)
    if "sub_range" in keys(param_dict)
        states = bosonstates(param_dict["projection"][1], param_dict["PN"])
        basis_mb = ManyBodyBasis(param_dict["projection"][1]yrange, states)
        param_dict["basis_mb"] = basis_mb
    else
        states = bosonstates(param_dict["basis"], param_dict["PN"])
        basis_mb = ManyBodyBasis(param_dict["basis"], states)
        param_dict["basis_mb"] = basis_mb
    end
    return basis_mb
end

get_basis_mb (generic function with 1 method)